# 🔥 How to use Focoos Pruning functionality

## 🐍 Setup Focoos

In [ ]:
%pip install 'focoos @ git+https://github.com/FocoosAI/focoos.git'

# 🎨 What is Pruning and how to prune

**Pruning** is a technique used to reduce the size and complexity of a neural network by removing less important parameters.

Pruning can target:

* **Weights:** Removing individual connections.
* **Neurons or filters:** Removing entire channels or feature maps (structured pruning).

Unlike the built-in `torch.nn.utils.prune module`, which only masks weights or entire layers (keeping their original size and structure), the `torch-pruning` library goes further. It actually removes the pruned weights or filters from the model architecture.

It can updates:

* The **layer dimensions**, and
* The **dependencies** between layers (so that input/output shapes remain consistent).

Focoos pruning functionality is **built on top of the `torch-pruning` library**, inheriting its core functionality and dependency graph logic.


# ⚙️ Focoos Pruning Setup

You can define which layers to prune in a specific model by providing their names as strings in a list. There are two main ways to do this:

- Manually create the list of layer names you wish to prune.

- Load a predefined list from `prunable_layers.json`, which contains recommended prunable layers for various model architectures.

For the selected layers, the pruning intensity is controlled using the `prune_ratio` parameter, which defines the fraction of channels to remove from each layer.

In [ ]:
from focoos import DatasetLayout, Task

TASK = Task.DETECTION
DATASET_NAME = "coco_2017_det"
DATASET_LAYOUT = DatasetLayout.CATALOG
DEVICE = "cpu"
RESOLUTION = 640
BENCHMARK_ITERATIONS = 200
VERBOSE = False
DO_EVAL = True  # compute or not eval metrics

OUTPUT_FOLDER = "pruning_outputs"  # folder to save the pruning results and pruned model

In [ ]:
import json
import os

from focoos import PACKAGE_DIR

layers_to_prune = json.load(open(os.path.join(PACKAGE_DIR, "pruning", "prunable_layers.json")))

MODEL_NAME = "fai-detr-m-coco"
PRUNE_RATIO = 0.3
LAYERS_TO_PRUNE = layers_to_prune[MODEL_NAME]

# 🚀 Setup and run pruning

In [ ]:
from focoos.pruning.focoos_pruning import FocoosPruning

pruning_pipeline = FocoosPruning(
    task=TASK,
    dataset_name=DATASET_NAME,
    dataset_layout=DATASET_LAYOUT,
    device=DEVICE,
    verbose=VERBOSE,
    do_eval=DO_EVAL,
    root_dir=OUTPUT_FOLDER,
    model_name=MODEL_NAME,
    resolution=RESOLUTION,
    prune_ratio=PRUNE_RATIO,
    benchmark_iterations=BENCHMARK_ITERATIONS,
    layers_to_prune=LAYERS_TO_PRUNE,
)

The Focoos pruning pipeline outputs a summary table that compares key metrics of the pruned model against the original one.

In [ ]:
pruning_pipeline.run()

## 🔁 Iterative Pruning and Re-training (beta)

The best practice is to prune **iteratively**, not all at once:

1. Prune a small percentage of weights or filters.
2. Retrain (or fine-tune) the model to recover accuracy.
3. Repeat until you reach the desired compression level.

This **gradual approach** preserves performance much better than aggressive one-shot pruning.

💡 Tip: Not all layers contribute equally to model efficiency or accuracy.
It’s often best to selectively prune layers that are over-parameterized or less critical to the model’s performance. Pruning all layers uniformly can cause unnecessary accuracy degradation. Selective pruning enables smarter and more effective compression.
At each pruning iteration, carefully evaluate which layers to prune, and avoid pruning the exact same layers in every cycle to maintain an adaptive optimization process.

⚠️ Work in progress: Currently, the process supports only a single automated pruning iteration.

### ⚙️ Iterative Pruning-Training Setup

In [ ]:
import os

from focoos import PACKAGE_DIR

layers_to_prune = json.load(open(os.path.join(PACKAGE_DIR, "pruning", "prunable_layers.json")))

TASK = Task.DETECTION
DATASET_NAME = "coco_2017_det"
DATASET_LAYOUT = DatasetLayout.CATALOG
DEVICE = "cpu"
RESOLUTION = 640
BENCHMARK_ITERATIONS = 200
VERBOSE = False  # print or not additional pruning information
DO_EVAL = False  # compute or not eval metrics

MODEL_NAME = "fai-detr-m-coco"
OUTPUT_FOLDER = "pruning_outputs-test"

PRUNE_RATIO = 0.5
LAYERS_TO_PRUNE = layers_to_prune[MODEL_NAME]

In [ ]:
from focoos import DatasetLayout, Task, TrainerArgs
from focoos.data import AutoDataset, get_default_by_task
from focoos.ports import DatasetSplitType
from focoos.pruning.focoos_pruning import FocoosPruning

pruning_pipeline = FocoosPruning(
    task=TASK,
    dataset_name=DATASET_NAME,
    dataset_layout=DATASET_LAYOUT,
    device=DEVICE,
    verbose=VERBOSE,
    do_eval=DO_EVAL,
    root_dir=OUTPUT_FOLDER,
    model_name=MODEL_NAME,
    resolution=RESOLUTION,
    prune_ratio=PRUNE_RATIO,
    benchmark_iterations=BENCHMARK_ITERATIONS,
    layers_to_prune=LAYERS_TO_PRUNE,
)

auto_dataset = AutoDataset(
    dataset_name="coco_2017_det",
    task=Task.DETECTION,
    layout=DatasetLayout.CATALOG,
)

train_augs, val_augs = get_default_by_task(Task.DETECTION, resolution=RESOLUTION)

train_dataset = auto_dataset.get_split(augs=train_augs.get_augmentations(), split=DatasetSplitType.TRAIN)
val_dataset = auto_dataset.get_split(augs=val_augs.get_augmentations(), split=DatasetSplitType.VAL)

### 🚀 Run first iteration of Pruning-Training process

In [ ]:
from focoos import ModelManager

# 1st step: Pruning
pruning_pipeline.run()
output_directory = pruning_pipeline.output_directory

# 2nd step: Training

# Set pruning output directory in TrainerArgs
assert output_directory is not None
args = TrainerArgs(
    run_name=f"{MODEL_NAME}_{val_dataset.name}",
    batch_size=8,
    max_iters=10_000,
    eval_period=500,
    learning_rate=0.0008,
    device=DEVICE,
    output_dir=output_directory,
)

# Training
model = ModelManager.get(output_directory)
model.train(args, train_dataset, val_dataset)